In [11]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

from ast import literal_eval

from sklearn.metrics.pairwise import cosine_similarity

import requests

import gzip

from sklearn.feature_extraction.text import CountVectorizer

import sqlite3 as sql

from pydantic import BaseModel
from typing import Optional, List
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from sqlalchemy import Boolean, Column, Float, String, Integer
import uvicorn

from flask import Flask, flash, redirect, render_template, request, url_for
from flask_sqlalchemy import SQLAlchemy
import json

from bs4 import BeautifulSoup
import math

import aiohttp
import asyncio
from aiohttp import ClientSession

import nest_asyncio
# nest_asyncio.apply()
# __import__('IPython').embed()

## Reading data from url

In [4]:
%%time

url_list = ['https://datasets.imdbws.com/title.akas.tsv.gz', 'https://datasets.imdbws.com/title.basics.tsv.gz', 'https://datasets.imdbws.com/title.crew.tsv.gz','https://datasets.imdbws.com/title.principals.tsv.gz','https://datasets.imdbws.com/title.ratings.tsv.gz', 'https://datasets.imdbws.com/name.basics.tsv.gz']

def read_url(url):
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        r = requests.get(url)
        f.write(r.content)                  #save gz file
    with gzip.open(filename, 'rb') as f:    #open gz file and store into dataframe
        df = pd.read_csv(f,sep="\t")  
    return df

title_df = read_url(url_list[0])
title_df2 = read_url(url_list[1])
crew_df = read_url(url_list[2])
cast_df = read_url(url_list[3])
ratings_df = read_url(url_list[4])
names_df = read_url(url_list[5])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 2min 11s, sys: 21.2 s, total: 2min 32s
Wall time: 4min 20s


Since reading data takes only around 4 minutes and we only need to run this once in a while, it should be fine without optimization

## Preprocessing

In [5]:
def process_duration(duration):
    try:
        return int(duration)
    except:
        return 0

def process_title(title):
    if title == 'tvMovie':
        return 'movie'
    elif title == 'tvEpisode' or title == 'tvMiniSeries':
        return 'tvSeries'
    else:
        return title

def process_string(string):           #split string into list, keep first 3 actors
    actor_list = string.split(',')
    if len(actor_list) > 3:
        actor_list = actor_list[:3]
    return actor_list

title_df_region = title_df.loc[title_df.region.isin(['US', 'GB', 'TW', 'CN', 'HK', 'KR'])]
title_df_region = title_df_region.drop_duplicates(subset=['titleId'])
title_df2['startYear'] = title_df2['startYear'].apply(process_duration)
title_df2['runtimeMinutes'] = title_df2['runtimeMinutes'].apply(process_duration)
title_df2_startYear = title_df2.loc[title_df2['startYear'] >= 2000]
combined_df = title_df_region.merge(title_df2_startYear, left_on = 'titleId', right_on = 'tconst').drop(['tconst'],axis=1).merge(ratings_df, left_on = 'titleId', right_on = 'tconst').drop(['tconst'],axis=1)
combined_df2 = combined_df.loc[combined_df['titleType'].isin(['movie','tvMovie','tvEpisode','tvSeries','tvMiniSeries'])]
combined_df2['titleType'] = combined_df2['titleType'].apply(process_title)
crew_df_combined = crew_df.merge(names_df, left_on = 'directors', right_on = 'nconst').drop(['nconst'],axis = 1)
directors = crew_df_combined[['tconst', 'directors', 'primaryName']]
directors2 = directors.loc[directors.tconst.isin(combined_df2.titleId)==True] 
cast_df_combined = cast_df.loc[cast_df.category.isin(['actor','actress'])].merge(names_df, left_on = 'nconst', right_on = 'nconst')
actors = cast_df_combined[['tconst', 'nconst', 'primaryName']]
actors2 = actors.loc[actors.tconst.isin(combined_df2.titleId)==True]    #filter by selected movies
actors_string_nconst = actors2.groupby('tconst')['nconst'].apply(lambda x: ','.join(x)).reset_index()
actors_string_name = actors2.groupby('tconst')['primaryName'].apply(lambda x: ','.join(x)).reset_index()
actors_string_nconst['nconst'] = actors_string_nconst['nconst'].apply(process_string)
actors_string_name['primaryName'] = actors_string_name['primaryName'].apply(process_string)
actors_df = actors_string_nconst.merge(actors_string_name, on = 'tconst')
final_df = combined_df2.merge(actors_df, how = 'left', left_on = 'titleId', right_on = 'tconst').drop(['tconst'],axis=1).rename(columns = {'nconst': 'actor_id', 'primaryName': 'actor_name'}, inplace = False)
final_df2 = final_df.merge(directors2, how = 'left', left_on = 'titleId', right_on = 'tconst').drop(['tconst'],axis=1).rename(columns = {'directors': 'director_id', 'primaryName': 'director_name', 'titleId': 'title_id', 'titleType': 'movie_type'}, inplace = False)
final_df3 = final_df2[['title_id', 'title', 'region', 'movie_type', 'genres', 'actor_id', 'actor_name', 'director_id', 'director_name', 'startYear', 'runtimeMinutes', 'averageRating', 'numVotes']]
final_df4 = final_df3.query('averageRating >= 5')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
%%time

final_df4_sim = final_df4.iloc[0:1000].copy()

# Function to convert all strings to lower case and strip names of spaces, as well as list
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
def clean_genres(genre_str):        #for loop required, since need to replace 'Action' and 'Adventure' at the same time
    if genre_str.lower() == '\n' or genre_str.lower() == '\\n':
        genre_str = ""
    genre_str = genre_str.replace("Short", "")
    genre_str = genre_str.replace("Western", "")
    genre_str = genre_str.replace("Adult", "")
    genre_str = genre_str.replace("War", "")
    
    if genre_str != "":
        genre_list = []
        genre_str = genre_str.split(',')
        for genre in genre_str:
            if genre == 'Action' or genre == 'Adventure':
                genre_list.append('Action & Adventure')
            elif genre == 'Sci-Fi' or genre == 'Fantasy':
                genre_list.append('Sci-Fi & Fantasy')
            elif genre == 'Musical':
                genre_list.append('Music')
            elif genre == 'Biography' or genre == 'History' or genre == 'News':
                genre_list.append('Documentary')
            elif genre == 'Game-Show' or genre == 'Talk-Show' or genre == 'Reality-TV':
                genre_list.append('Reality TV and Talk shows')
            elif genre == '':
                pass
            else:
                genre_list.append(genre)
        genre_list = list(set(genre_list))
    else:
        genre_list = genre_str
    return genre_list

final_df4_sim['actor_name_clean'] = final_df4_sim['actor_name'].apply(clean_data)
final_df4_sim['director_name_clean'] = final_df4_sim['director_name'].apply(clean_data)
final_df4_sim['genres_list'] = final_df4_sim['genres'].apply(clean_genres)
final_df4_sim['genres_list_lower'] = final_df4_sim['genres_list'].apply(lambda x: [genre.lower() for genre in x])
final_df4_sim = final_df4_sim.loc[final_df4_sim.genres_list != ""]

CPU times: user 11.2 ms, sys: 16.5 ms, total: 27.7 ms
Wall time: 41.8 ms


## Scraping overview, image and keywords

The data we requested from IMDB website doesn't have overview, image and keywords. We need the overview and image as they are important information we want to show users in our recommendations. Keywords may be important features for determining movie similarity as well.

Let's try several ways to scrape the data:

1) Using .apply and separate functions
   - This is very inefficient, since we're making requests multiple times to the same url
   - This took 8 mins 20 sec for 100 links.
   
2) Using loop over numpy array and one function
   - This is more efficient, since we're making request 1 time to the url to extract info
   - This took 6 mins for 100 links. But we still need to store the info in columns
   
3) Using Async IO
   - Using asychronous process instead of sequential to get speedup
   - This works very well and only took 30s for 100 links. But we still need to store the info in columns

In [68]:
%%time

def get_overview(row):
    try:
        url = "https://www.imdb.com/title/%s/" %(row.title_id) 
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        tag = soup.find('div', attrs={'class': 'ipc-html-content ipc-html-content--base'})         #seems to only work for unique classes
        return tag.getText()
    except:
        return ""

def get_image(row):
    try:
        title_name = row.title
        actors = row.actor_name_clean
        url = "https://www.imdb.com/title/%s/" %(row.title_id)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        images = soup.findAll('img') 
        for img in images:
            if 'Poster' in (img['alt']) or 'Trailer' in (img['alt']) or title_name in (img['alt']) or actors[0] in (img['alt']):    #actor_list should have at least 1 element
                image_final = img['src']
                return image_final
    except:
        return ""
        
def get_keywords(row):
    try:
        url = "https://www.imdb.com/title/%s/keywords" %(row.title_id)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        tag = soup.findAll('div', attrs={'class': 'sodatext'})
        keyword_list = []
        for keyword in tag[0:5]:
            keyword_list.append(keyword.getText().strip())
        return keyword_list
    except:
        return ""
 
final_df4_sim['overview'] = final_df4_sim.apply(get_overview, axis = 1)
final_df4_sim['image'] = final_df4_sim.apply(get_image, axis = 1)
final_df4_sim['keywords'] = final_df4_sim.apply(get_keywords, axis = 1)

CPU times: user 36.1 s, sys: 1.81 s, total: 37.9 s
Wall time: 8min 27s


In [116]:
%%time

def scrape_info(df_arr):
        title_id = df_arr[0]
        title_name = df_arr[1]
        actors = df_arr[6]

        url = "https://www.imdb.com/title/%s/" %(title_id) 
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')

        #Get overview
        tag = soup.find('div', attrs={'class': 'ipc-html-content ipc-html-content--base'})         #seems to only work for unique classes
        if tag != None:
            overview = tag.getText()
        else:
            overview = float('nan')
            
        #Get images
        images = soup.findAll('img') 
        image_final = float('nan')
        for img in images:
            if isinstance(actors,list):
                if 'Poster' in (img['alt']) or 'Trailer' in (img['alt']) or title_name in (img['alt']) or actors[0] in (img['alt']):    #actor_list should have at least 1 element
                    image_final = img['src']
            else:
                if 'Poster' in (img['alt']) or 'Trailer' in (img['alt']) or title_name in (img['alt']):    #actor_list should have at least 1 element
                    image_final = img['src']

        #Get keywords
        url = "https://www.imdb.com/title/%s/keywords" %(title_id)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        tag = soup.findAll('div', attrs={'class': 'sodatext'})
        keyword_list = []
        for keyword in tag[0:5]:      
            keyword_list.append(keyword.getText().strip())
        
        return (overview, image_final, keyword_list)

title_df = {}
title_array = final_df4_sim.to_numpy()
for title in title_array:
    title_df[title[0]] = scrape_info(title)    #store dict with format {title_id: (overview, image, keyword_list)}

CPU times: user 22.2 s, sys: 994 ms, total: 23.2 s
Wall time: 6min 2s


In [10]:
%%time

async def scrape_info(session, df_arr):
        title_id = df_arr[0]
        title_name = df_arr[1]
        actors = df_arr[6]
        
        
        url = "https://m.imdb.com/title/%s/" %(title_id)
        async with session.get(url) as response:
            response_text = await response.text()       #need to use response.text() instead of response.text which is an object instead of function
            soup = BeautifulSoup(response_text, 'lxml')

            #Get overview
            tag = soup.find('div', attrs={'class': 'ipc-html-content ipc-html-content--base'})         #seems to only work for unique classes
            if tag != None:
                overview = tag.getText()
            else:
                overview = float('nan')

            #Get images
            images = soup.findAll('img') 
            image_final = float('nan')
            for img in images:
                if isinstance(actors,list):
                    if 'Poster' in (img['alt']) or 'Trailer' in (img['alt']) or title_name in (img['alt']) or actors[0] in (img['alt']):    #actor_list should have at least 1 element
                        image_final = img['src']
                else:
                    if 'Poster' in (img['alt']) or 'Trailer' in (img['alt']) or title_name in (img['alt']):    #actor_list should have at least 1 element
                        image_final = img['src']

        #Get keywords
        url_kw = "https://m.imdb.com/title/%s/keywords" %(title_id)
        async with session.get(url_kw) as response_kw:
            response_kw_text = await response_kw.text()
            soup_kw = BeautifulSoup(response_kw_text, 'lxml')
            tag_kw = soup_kw.findAll('div', attrs={'class': 'sodatext'})
            keyword_list = []
            for keyword in tag_kw[0:5]:      
                keyword_list.append(keyword.getText().strip())
        return (title_id, overview, image_final, keyword_list)
    
async def main(): 
    title_array = final_df4_sim.to_numpy()       
    async with ClientSession() as session:
        title_list = await asyncio.gather(*[scrape_info(session, title) for title in title_array])
        return title_list
    
title_list = asyncio.run(main())

CPU times: user 18.9 s, sys: 1.89 s, total: 20.8 s
Wall time: 25.9 s


In [152]:
scraped_df = pd.DataFrame(title_list, columns=['title_id', 'overview', 'image', 'keywords'])   #convert list of tuples to df
final_df4_sim2 = final_df4_sim.merge(scraped_df, how = 'left', on = 'title_id') 

## Computing similarity and identifying most similar movies

In [153]:
def create_soup(x):
    return x['region'] + ' ' + x['movie_type'] + ' ' + ' '.join(x['actor_name_clean']) + ' ' + x['director_name_clean'] + ' ' + ' '.join(x['genres_list_lower']) + ' ' + ' '.join(x['keywords'])

final_df4_sim2['soup'] = final_df4_sim2.apply(create_soup, axis=1)

In [154]:
final_df4_sim2['soup'][0]

'CN movie xuepengfan yi-minwen shaoquanzhu yi-minwen action & adventure justice'

In [155]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(final_df4_sim2['soup'])
count_matrix.shape

(1000, 5586)

In [156]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [157]:
#Since movies may not go in order 
final_df4_sim2 = final_df4_sim2.reset_index(drop = True)

#Create Series where index is title_id, and column is index
indices = pd.Series(final_df4_sim2.index, index=final_df4_sim2['title_id'])   

def insert_top_movies(row):
    # Get the index of the movie that matches the title
    idx = indices[row.title_id]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))      #creates list of tuple (index, similarity score)

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 30 most similar movies
    sim_scores = sim_scores[1:31]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    #Return top 10 movies with highest rating
    top_movies = final_df4_sim2.iloc[movie_indices].sort_values(by = 'averageRating', ascending = False)[['title_id','title','image']][0:10] 
    top_movies_tuple = list(zip(top_movies.title_id, top_movies.title, top_movies.image))
    return top_movies_tuple

final_df4_sim2['top_movies'] = final_df4_sim2.apply(insert_top_movies, axis = 1)

## Load data into movie database

In [179]:
str_columns = ['title_id', 'title', 'region', 'movie_type', 'genres', 'director_id', 'director_name', 'director_name_clean', 'overview', 'image', 'soup']
list_columns = ['actor_id', 'actor_name', 'actor_name_clean', 'genres_list', 'genres_list_lower', 'keywords', 'top_movies']

for column in str_columns:
    final_df4_sim2[column] = final_df4_sim2[column].astype(str) 

for column in list_columns:
    final_df4_sim2[column] = final_df4_sim2[column].apply(lambda x: json.dumps(x))     #SQLite3 doesn't accept array. Cannot use .to_json() on entire column, since will return same value for all rows

movie_db = sql.connect('movie_recsys4.db')
final_df4_sim2.to_sql('movie', movie_db)

Now that we have the database containing the list of movies, essential information and most similar movies, we can create our Flask app to interact with the user